In [ ]:
!pip install PyPDF2

In [ ]:
#read pdf file
from pypdf import PdfReader


reader = PdfReader("AshlyLauResume.pdf")
number_of_pages = len(reader.pages)
page = reader.pages[0]
text = page.extract_text().replace("to", "To").replace("•", "")

#text = remove_punctuation(text)


In [2]:
import re
import spacy

nlp = spacy.load('en_core_web_sm')

def extract_name(resume_text):
    nlp_text = nlp(resume_text)
    first_name = ""
    last_name = ""
    for token in nlp_text:
        if token.pos_ == 'PROPN' and not token.is_stop:
            
            if first_name == "":
                first_name = token.text
            else:
                last_name = token.text
    
    return f"{first_name} {last_name}"

email_pattern = r'[\w\.-]+@[a-zA-Z\d\.-]+\.[a-zA-Z]{2,}'
phone_pattern = r'(\+\d{1,2}\s?)?(\(?\d{3}\)?[\s.-]?)?\d{3}[\s.-]?\d{4}'

def extract_email(text):
    return re.findall(email_pattern, text)

def extract_phone_number(text):
    return re.findall(phone_pattern, text)

print(extract_name(text))
print(extract_email(text))
print(extract_phone_number(text))


Ashly Service
['ashlylau@gmail.com']
[]


In [3]:
#Extracting Degree:

#Defines a function extract_degree to extract degrees (Bachelor, Master, PhD, Associate) from the text using regular expressions.
import re

def extract_degree(text):
  pattern = r'(?=Bachelor|Master|Phd|Associate)(\w+\s).*'

  matches = re.search(pattern, text)
  if matches:
    return matches.group(0)
print(extract_degree(text))

None


In [4]:
#all universities https://www.usnews.com/education/best-global-universities/search?format=json&page=4

#Extracting University Name:
#Reads a CSV file containing a list of university names and defines a function find_university_name to extract university names from the text using regular expressions.

import re
import pandas as pd
df = pd.read_csv("all_universities.csv")

university_list = df["name"].tolist()


def find_university_name(resume_text, universities):
    found_universities = []
    #resume_text = resume_text.lower()
    for university in universities:
        #university = university.replace("&", "and")
        pattern = r'\b{}\b'.format(re.escape(university))  # Create a regex pattern for each university
        if re.search(pattern, resume_text, re.IGNORECASE):
            found_universities.append(university)
    return found_universities # list of founded univ
print(find_university_name(text, university_list))

['Imperial College London']


In [5]:
#Extracting Skills:

#Reads a CSV file containing a list of skills and defines a function extract_skills to extract skills from the text using regular expressions.

import pandas as pd
import re

df = pd.read_csv("skills.csv")
skills_list = df["0"].tolist()

def extract_skills(resume_text, skills):
    found_skills = []
    for university in skills:
        pattern = r'\b{}\b'.format(re.escape(university))  # Create a regex pattern for each university
        if re.search(pattern, resume_text, re.IGNORECASE):
            found_skills.append(university.lower())
    return set(found_skills)
print(extract_skills(text, skills_list))

{'postgresql', 'economics', 'html5', 'facebook', 'javascript', 'database', 'chemistry', 'cisco', 'marketing', 'c', 'engineering', 'mobile', 'operating systems', 'workflow', 'scheduling', 'teaching', 'segmentation', 'real-time', 'machine learning', 'django', 'java', 'physics', 'ux', 'github', 'gmail', 'analysis', 'mathematics', 'r', 'data analysis', 'tensorflow', 'python', 'android', 'coding', 'data collection', 'video', 'international', 'ai', 'filing', 'programming'}


In [6]:
import nltk
from itertools import permutations
from nltk.tokenize import word_tokenize
import string
from ordered_set import OrderedSet
import spacy


# Download required NLTK resources
#nltk.download('averaged_perceptron_tagger')
#nltk.download('punkt')

# Define helper functions

#Text Preprocessing:
#Removes punctuation, tokenizes the text, and performs part-of-speech tagging using NLTK.

def remove_punctuation(text):
    """
    Remove punctuation from the text and return the cleaned text.
    """
    text = text.replace("–", "to").replace("-", "to")
    words = word_tokenize(text)
    words_without_punctuation = [word for word in words if word not in string.punctuation]
    return ' '.join(words_without_punctuation)

def remove_verb_plus(text):
    """
    Remove verbs and other specified parts of speech, returning cleaned text.
    """
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    strr = ""
    for i in pos_tags:
        if i[1] in ["NNP", "CD", "TO"]:
            strr += i[0] + " "
    return strr

def remove_repetitive_words(txt):
  result = OrderedSet(txt.split(" "))
  g = " ".join(result)
  return g

#Extracting Organizations:
#Uses spaCy's named entity recognition (NER) to extract organization names from the text.

def extract_organizations(text):
    """
    Extract organization names from the text using spaCy's named entity recognition (NER).
    """
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    organizations = [ent.text for ent in doc.ents if ent.label_ == "ORG"]
    return organizations

def find_org_in_text(text, orgs):
    """
    Find and return organization names present in the given text.
    """
    new_list = []
    for org in orgs:
        if org in text:
            new_list.append(org)
    return new_list



#Searching for Patterns:
#Defines a function search to search for specific patterns in the text based on part-of-speech tags.

def search(pat):
    """
    Search for patterns in the POS-tagged text and extract relevant information.
    """
    M = len(pat)
    N = len(pos_tags)

    results = []
    for i in range(N-M):
        for j in range(M):
            k = j + 1
            if pos_tags[i+j][1] != pat[j]:
                break
        if k == M: # found pat tern

            index = pat.index("CD") # to split text into []"CD"[] , the left side will contain an unstrucutred organization , so we will format it with the list of orgs we have before

            left_text = " ".join([t[0] for t in pos_tags[i-5:i+index-1]])

            orgs_found_in_left = find_org_in_text(remove_verb_plus(left_text), all_orgs_in_cv)

            if len(orgs_found_in_left) == 0:
                results.append(" ".join([t[0] for t in pos_tags[i:i+M+1]]))
            else:
                orgs_found_in_left = sorted(orgs_found_in_left, key=len, reverse=True)[0]
                final_text = orgs_found_in_left + " " + " ".join([t[0] for t in pos_tags[i+index-1:i+M+1]])
                results.append(final_text)

            del pos_tags[i:i+M]#delete the found text to not get the same text / similar text with 2 different patterns , so we get unique results

            N = len(pos_tags) #since we delete text , we should update the length
            M = M - i
    return results
def remove_text_from_text(text1, text2):
  text1_list = text1.split(" ")
  for item in text1_list:
    text2 = text2.replace(item, "")
  return text2

#
text = remove_punctuation(text)

pos_tags = nltk.pos_tag(text.split())

# Define patterns to search for in the text
patterns = [
    ("NNP", "NNP", "NNP", "NNP", "NNP", "NNP", "CD", "NNP", "NNP"),
    ("NNP", "NNP", "NNP", "NNP", "NNP", "CD", "NNP", "NNP"),
    ("NNP", "NNP", "NNP", "NNP", "NNP", "CD", "NNP", "NNP"),
    ("NNP", "NNP", "NNP", "NNP", "NNP", "CD", "TO", "CD", "CD"),
    ("NNP", "NNP", "NNP", "NNP", "NNP", "CD", "TO", "NNP"),
    ("NNP(, ""NNP", "NNP", "NNP", "NNP", "CD", "TO", "NNP", "CD"),
    ("NNP", "NNP", "NNP", "NNP", "NNP", "NNP", "CD", "TO", "NNP", "CD"),
    ("NNP", "NNP", "NNP", "NNP", "NNP", "CD", ":", "NNP", "NNP", "NNP"),
    ("NNP", "NNP", "NNP", "NNP", "NNP", "CD", ":", "NNP", "NNP"),
    ("NNP", "NNP", "NNP", "NNP", "NNP", "CD", ":", "NNP"),
    ("NNP", "NNP", "NNP", "NNP", "NNP", "CD", ":", "CD"),
    ("NNP", "NNP", "NNP", "NNP", "NNP", "CD", "TO", "CD", "CD"),
    ("NNP", "NNP", "NNP", "CD", "TO", "CD"),
    ("NNP", "NNP", "NNP", "NNP", "NNP", "CD", "TO", "CD", "NNP", "NNP"),
    ("NNP", "NNP", "NNP", "NNP", "CD", "TO", "CD", "NNP", "NNP"),
    ("NNP", "NNP", "NNP", "NNP", "CD", "TO", "CD", "NNP", "NNP"),
    ("NNP", "NNP", "NNP", "NNP", "NNP", "CD", "TO", "CD", "NNP", "NNP"),
    ("NNP", "NNP", "NNP", "NNP", "NNP", "CD", "TO", "CD", "NNP"),
    ("NNP", "NNP", "NNP", "NNP", "NNP", "CD", "TO", "CD", "CD"),
    ("NNP", "NNP", "NNP", "NNP", "NNP", "CD", ":", "NNP", "NNP", "NNP"),
    ("NNP", "NNP", "NNP", "NNP", "NNP", "CD", ":", "CD", "NNP", "NNP"),
    ("NNP", "NNP", "NNP", "NNP", "NNP", "CD", ":", "CD", "NNP", "NNP"),
    ("NNP", "NNP", "NNP", "NNP", "NNP", "CD", ":", "CD", "NNP"),
    ("NNP", "NNP", "NNP", "NNP", "NNP", "CD", ":", "CD"),
    ("NNP", "NNP", "NN", "NNP", "NNP", "CD", "TO", "CD", "NNP", "NNP"),
    ("NNP", "NNP", "NN", "NNP", "NNP", "CD", ":", "CD", "NNP", "NNP"),
    ("NNP", "NNP", "NNP", "NNP","NNP", "CD", "TO", "CD", "CD"),
    ("NNP", "NNP", "NNP", "NNP","NNP", "CD", "HYPH", "CD"),
    ("NNP", "NNP", "CD", "HYPH", "NNP", "NNP", "NNP"),
    ("NNP", "CD", "HYPH", "CD", "NNP", "NNP"),
    ("NNP", "CD", ":", "NN", "NNP", "NNP"),
    ("NNP", "NNP", "NNP", "NNP", "CD", "TO", "NNP", "NNP"),

    ("NNP", "NNP", "CD", "TO", "NNP", "NNP"),
]

for i in range(len(patterns)-1):
  patterns.append(patterns[i][::-1])

# Sort patterns by length in descending order
sorted_patterns = sorted(patterns, key=len, reverse=True)

# Extract organization names from the text
all_orgs_in_cv = extract_organizations(text)

# Search for patterns in the POS-tagged text

final_res = []
for pp in sorted_patterns:

    res = search(pp)
    if len(res) != 0:
      for r in res:
        #s = remove_repetitive_words(r)
        print("---" + r)
        text = remove_text_from_text(r, text)



---Raffles Institution 01/2015 to 12/2016 CambridgetoSingapore GCE A Level · 7
---EXPERIENCE Google 07/2019 to 09/2019 Summer Trainee Engineering Practicum Intern ·
---Android Bootstrap 3 TensorFlow AWARDS/ACHIEVEMENTS ·
---Singapore National Team Artistic Gymnastics 01/2012 to 12/2016 · Participated
---Glasgow Commonwealth Games PROJECTS CHARJE 10/2019 to 01/2020 · Implemented
---DDB Worldwide Pte Ltd 02/2017 to 04/2017 Account Executive Intern
---Android Council 01/2015 to 12/2016 · EAGLES
---HP Inc. 07/2018 to 08/2018 Software R D Intern ·
---Imperial College London 09/2017 to 06/2021 Computing


In [ ]:
#Extracting Key Achievements:
#Uses spaCy for named entity recognition to extract key achievements from the text.

import spacy
nlp = spacy.load('en_core_web_sm')
text = sorted(text.split("Experience"), key=len, reverse=True)[0]
for org in all_orgs_in_cv:
  text = text.replace(org, "").replace("•", "")
doc = nlp(text)
index = []

for i in range(len(doc)-3):
    if doc[i].pos_ == "VERB" and doc[i-1].pos_ not in ["PART", "PRON", "ADP", "VERB", "DET"] and doc[i-1].text != "and" and doc[i].text[0].isupper():
        index.append(i)

    if doc[i].pos_ == "PROPN" and doc[i+1].pos_ == "NOUN" and doc[i-1].pos_ not in ["PART", "PRON", "DET", "ADP", "VERB"] and doc[i].text[0].isupper():
        index.append(i)

    if doc[i].pos_ == "PROPN" and doc[i+1].pos_ == "ADJ" and doc[i+2].pos_ == "NOUN" and doc[i-1].pos_ not in ["PART", "PRON", "DET", "ADP", "VERB"] and doc[i].text[0].isupper():
        index.append(i)
    if doc[i].pos_ == "PROPN" and doc[i+1].pos_ == "ADP" and doc[i+2].pos_ == "NOUN" and doc[i-1].pos_ not in ["PART", "PRON", "DET", "ADP", "VERB"] and doc[i].text[0].isupper():
        index.append(i)
    if doc[i].pos_ == "PRON" and doc[i+1].pos_ == "AUX" and doc[i-1].pos_ not in ["PART", "PRON", "DET", "ADP", "VERB"] and doc[i].text[0].isupper():
        index.append(i) # like : I was ..

for i in range(len(index)-1):
    print(doc[index[i]:index[i+1]])
    print("___")




In [ ]:
#Extracting Universities from Website:

#Scrapes university names from the "usnews.com" website and saves them to a CSV file.

import requests
import pandas as pd
import json


resutls = []

for i in range(217):
  url = f'https://www.usnews.com/education/best-global-universities/search?format=json&page={i}'
  headers = {
      'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:126.0) Gecko/20100101 Firefox/126.0'
  }

  response = requests.get(url, headers=headers)
  if response.status_code == 200:
      print("Request successful")
      js = json.loads(response.text)
      items = js.get("items", "")
      for i in items:
        name = i.get("name", "")
        resutls.append(name)
      # Process response here
  else:
      print("Request failed with status code:", response.status_code)

df = pd.DataFrame(resutls)
df.to_csv("all_universities.csv")

In [ ]:
#Extracting skills from dataset:
from bs4 import BeautifulSoup
import lxml
import nltk
from itertools import permutations
from nltk.tokenize import word_tokenize
import string
import pandas
import os


# Download required NLTK resources
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')


def extract_skills(text_html):
    soup = BeautifulSoup(text_html, 'lxml')
    t = soup.find(attrs={"data-section-cd":"HILT"})
    return t.text

skills = []

for filename in os.listdir(os.getcwd()):
    if os.path.isfile(os.path.join(os.getcwd(), filename)):
        with open(filename, 'r') as f:
            content = f.read()
            try:
                sk = extract_skills(content)
                skills.append(sk)
            except:
                pass


# Define helper functions
def remove_punctuation(text):
    """
    Remove punctuation from the text and return the cleaned text.
    """
    text = text.replace("–", "to").replace("-", "to")
    words = word_tokenize(text)
    words_without_punctuation = [word for word in words if word not in string.punctuation]
    return ' '.join(words_without_punctuation)

new_list = set()
for ele in skills:
    if len(ele) <= 50:
        ele = remove_punctuation(ele)
        for sub_ele in ele.split():
            new_list.add(sub_ele)


g = pandas.DataFrame(new_list)
g.to_csv("all_skills.csv")



In [19]:
#Using chatgpt

#Uses Selenium to interact with a website (chatgpt.com) for further processing, likely for generating a report based on extracted information

import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import json
from selenium import webdriver
# Initialize the undetected Chrome driver


driver = uc.Chrome()
prompt = '''
extract "{  "name": "","email": "" "phone": "", "education": [{"institution": "", "location": "", "year": ""  }],"experience": [  { "title": "", "company": "","location": "","dates": "","short_description": "", "key_acheviement": "", },]"certificates":"","skills":[]" from "''' + text + '" in json format , if some variables does not exist , put none'

try:
    driver.get('https://chatgpt.com/')
    
    time.sleep(10)
    # Print the title of the webpage
    text_area = driver.find_element(By.ID, "prompt-textarea")
    text_area.send_keys(prompt)
    time.sleep(4)
    text_area.send_keys(Keys.ENTER)
    time.sleep(40)
    code = driver.find_element(By.TAG_NAME, "code")
    c = json.loads(code.text)
    print(c)

finally:
    driver.quit()


{'name': 'shly Lau', 'email': 'ashlylau@gmail.com', 'phone': '', 'education': [{'institution': 'EUCTION', 'location': '', 'year': '09/201'}, {'institution': 'Cambridge', 'location': '', 'year': '0/2019'}], 'experience': [{'title': 'Account', 'company': '', 'location': '', 'dates': '02/201 - 04/201', 'short_description': 'ssisted with the formulation of marketing strategies by working with a team To brief media and creative staff  Facilitated communications between clients and creatives by managing client accounts and projects', 'key_acheviement': ''}], 'certificates': '', 'skills': ['Java', 'Python', 'Haskell', 'C', 'C++', 'JavaScript', 'Prolog', 'HTML5', 'CSS', 'Android Development', 'Web Development', 'GitHub', 'React Native']}
